In [1]:
import spacy
import networkx as nx
import matplotlib.pyplot as plt
import os

In [2]:
# Charger le modèle spaCy approprié pour la langue
nlp = spacy.load("fr_core_news_sm")  # Chargez le modèle français



In [3]:
with open('../data/corpus_ASIMOV_leaderboard/corpus_asimov_leaderboard/prelude_a_fondation/chapter_2.txt.preprocessed', 'r', encoding='utf-8') as file:
    text = file.read()

In [5]:
from spacy import displacy
text = nlp(text)
displacy.render(text,style="ent",jupyter=True)

In [6]:
import os
import csv
import spacy
import networkx as nx

#nlp = spacy.load("fr_core_news_sm")

def ner_graph_lca(file):
    file_name = os.path.splitext(os.path.basename(file))[0]  # extraction du nom du fichier

    # Lire le contenu du fichier .txt
    with open(file, "r", encoding="utf-8") as file:
        text = file.read()

    # Augmenter la limite de longueur de texte
    nlp.max_length = len(text) + 100  # Augmentez la limite de 100 caractères pour être sûr

    # Analyse du texte avec spaCy
    doc = nlp(text)

    # Initialiser une liste pour stocker les entités nommées de type "PER"
    characters = [ent.text for ent in doc.ents if ent.label_ == "PER" and not any(token.pos_ == "PRON" for token in ent)]

    # Initialiser un dictionnaire pour stocker les distances entre les entités nommées
    distance_dict = {}

    # Parcourir le document pour calculer les distances entre les entités nommées
    for i in range(len(doc)):
        if doc[i].text in characters:
            for j in range(i + 1, min(i + 26, len(doc))):
                if doc[j].text in characters:
                    # Calculer la distance entre les entités nommées
                    distance = j - i - 1

                    # Assurer que la distance ne dépasse pas 25
                    distance = min(distance, 25)

                    # Créer une clé unique pour la paire d'entités nommées
                    key = tuple(sorted([doc[i].text, doc[j].text]))

                    # Ajouter la distance au dictionnaire
                    distance_dict[key] = distance

    # Créer un graphe avec NetworkX
    G = nx.Graph()

    # Ajouter les nœuds (entités nommées)
    G.add_nodes_from(characters)

    # Ajouter les arêtes (relations) avec les distances comme attributs
    for (char1, char2), distance in distance_dict.items():
        G.add_edge(char1, char2, distance=distance)

    # Enregistrer le graphe au format GEXF
    nx.write_gexf(G, f"../fig/lca/{file_name}.gexf")

    # Enregistrer les entités nommées dans un fichier CSV
    csv_file_path = f"../fig/lca/{file_name}_PER.csv"
    with open(csv_file_path, 'w', newline='', encoding='utf-8') as csv_file:
        csv_writer = csv.writer(csv_file)
        csv_writer.writerow(['Person'])  # Header
        csv_writer.writerows([[char] for char in characters])

    return csv_file_path


In [7]:
import os
import csv
import spacy
import networkx as nx

nlp = spacy.load("fr_core_news_sm")

def ner_graph_paf(file):
    file_name = os.path.splitext(os.path.basename(file))[0]  # extraction du nom du fichier

    # Lire le contenu du fichier .txt
    with open(file, "r", encoding="utf-8") as file:
        text = file.read()

    # Augmenter la limite de longueur de texte
    nlp.max_length = len(text) + 100  # Augmentez la limite de 100 caractères pour être sûr

    # Analyse du texte avec spaCy
    doc = nlp(text)

    # Initialiser une liste pour stocker les entités nommées de type "PER"
    characters = [ent.text for ent in doc.ents if ent.label_ == "PER" and not any(token.pos_ == "PRON" for token in ent)]

    # Initialiser un dictionnaire pour stocker les distances entre les entités nommées
    distance_dict = {}

    # Parcourir le document pour calculer les distances entre les entités nommées
    for i in range(len(doc)):
        if doc[i].text in characters:
            for j in range(i + 1, min(i + 26, len(doc))):
                if doc[j].text in characters:
                    # Calculer la distance entre les entités nommées
                    distance = j - i - 1

                    # Assurer que la distance ne dépasse pas 25
                    distance = min(distance, 25)

                    # Créer une clé unique pour la paire d'entités nommées
                    key = tuple(sorted([doc[i].text, doc[j].text]))

                    # Ajouter la distance au dictionnaire
                    distance_dict[key] = distance

    # Créer un graphe avec NetworkX
    G = nx.Graph()

    # Ajouter les nœuds (entités nommées)
    G.add_nodes_from(characters)

    # Ajouter les arêtes (relations) avec les distances comme attributs
    for (char1, char2), distance in distance_dict.items():
        G.add_edge(char1, char2, distance=distance)

    # Enregistrer le graphe au format GEXF
    nx.write_gexf(G, f"../fig/paf/{file_name}.gexf")

    # Enregistrer les entités nommées dans un fichier CSV
    csv_file_path = f"../fig/paf/{file_name}_PER.csv"
    with open(csv_file_path, 'w', newline='', encoding='utf-8') as csv_file:
        csv_writer = csv.writer(csv_file)
        csv_writer.writerow(['Person'])  # Header
        csv_writer.writerows([[char] for char in characters])

    return csv_file_path


In [8]:
for i in range(18):
    file_path = f"../data/corpus_ASIMOV_leaderboard/corpus_asimov_leaderboard/les_cavernes_d_acier/chapter_{i+1}.txt.preprocessed"
    globals()[f"chap_{i+1}"] = file_path
    ner_graph_lca(file_path)

In [9]:
for i in range(19):
    file_path = f"../data/corpus_ASIMOV_leaderboard/corpus_asimov_leaderboard/prelude_a_fondation/chapter_{i+1}.txt.preprocessed"
    globals()[f"chap_{i+1}"] = file_path
    ner_graph_paf(file_path)